# MTH 652: Advanced Numerical Analysis

## Homework Assignment 4

### <span style="color:red;">Write your name here</span>

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and code.
  * The use of ChatGTP, Copilot, or other AI assistants is **not allowed**
* The code must run in Colab or JupyterHub without errors.
  * Code that does not run will not receive any credit.
  * I suggest double-checking that your code runs properly in a new session. Sometimes code can be broken but appear to work because of old state in the notebook.

### Google Colab Instructions

* After opening this assignment in Google Colab, click on **"Copy to Drive"**
* Rename the notebook to `student_name_mth_652_assignment_4.ipynb`
    * ⚠️ In the above, replace `student_name` with your name!
* Enter your name above (in the cell below "Homework Assignment")!
* When you are ready to submit your assignment, select "File -> Download -> Download .ipynb" from the Colab menu
* Upload the downloaded `.ipynb` file to Canvas

### Assignment Goals

* The purpose of this assignment is to:
    1. Understand the theory of subspace correction methods
    2. Implement and study a simple two-level additive subspace correction method

## Written Questions

#### 1. (5 points)

Prove the following proposition from Lecture 13.

Let $A$ and $M$ be SPD matrices.
Then, the following eigenvalue problems have the same eigenvalues
\begin{align*}
    A \bm x &= \lambda M \bm x, \\
    M^{-1} A \bm x &= \lambda \bm x, \\
    M^{-1/2} A M^{-1/2} \bm x &= \lambda \bm x, \\
    A^{1/2} M^{-1} A^{1/2} \bm x &= \lambda \bm x.
\end{align*}
In all cases, the eigenvalues are real and positive.
The extremal eigenvalues satisfy
\begin{equation}
    \lambda_{\min}(M^{-1}A) = \inf_{\bm x} \frac{\bm x^T A \bm x}{\bm x^T M \bm x}, \qquad
    \lambda_{\max}(M^{-1}A) = \sup_{\bm x} \frac{\bm x^T A \bm x}{\bm x^T M \bm x}.
\end{equation}

## Coding Questions

In this assignment, we will implement a two-level domain decomposition preconditioner on structured grids.

The preconditioner will take the form
$$
    B = \sum_{i=0}^J I_i A_i^{-1} I_i^T,
$$
where $I_i : V_i \to V_h$ is the inclusion from the subspace $V_i$.

In our case, $V_h$ will be the standard piecewise linear finite element space on a Cartesian mesh with $n_x$ vertices in each dimension ($n_x$ odd).

The coarse space $V_0$ is defined on a Cartesian mesh with $(n_x + 1) / 2$ vertices in each dimension.
Note that the mesh of $V_h$ can be obtained from the mesh of $V_0$ through one uniform refinement.

For each basis function $\phi_i \in V_h$, the one-dimensional space $V_i$ is given by $V_i = \operatorname{span}\{ \phi_i \}$.

We will use the same finite element code we have been working with in this course (included below).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def area(K):
    """
    Returns the area of the triangle defined by K.
    """
    M = np.array([[K[0,0], K[0,1], 1],
                  [K[1,0], K[1,1], 1],
                  [K[2,0], K[2,1], 1]])
    return 0.5 * np.linalg.det(M)

def make_stiffness(V, T, B):
    """
    Assembles the stiffness matrix on the mesh defined by (T, V). Eliminates the
    essential boundary conditions defined by B.
    """
    N = V.shape[0]
    A = np.zeros((N, N))

    for it in range(T.shape[0]):
        K = V[T[it,:],:]
        G1 = np.array([[1, 1, 1],
                       [K[0,0], K[1,0], K[2,0]],
                       [K[0,1], K[1,1], K[2,1]]])
        G2 = np.array([[0,0],[1,0],[0,1]])
        G = np.linalg.solve(G1, G2)
        A_K = area(K) * G @ G.T

        A[np.ix_(T[it,:], T[it,:])] += A_K

    A[B,:] = 0.0
    A[:,B] = 0.0
    for i in B:
        A[i,i] = 1.0

    return A

def square_mesh(nx):
    """
    Generates a triangular Cartesian mesh of the unit square with nx vertices in
    each dimension.

    Returns (V, T, B), where V is are the vertex coordinates, T are the triangle
    indices, and B is a list of boundary vertex indices.
    """
    x = np.linspace(0, 1, nx)
    X, Y = np.meshgrid(x, x)
    V = np.stack((X.ravel(), Y.ravel()), axis=1)

    nt = 2*(nx-1)**2
    T = np.zeros((nt, 3), int)

    for iy in range(nx - 1):
        for ix in range(nx - 1):
            v1 = ix + iy*nx
            v2 = ix + 1 + iy*nx
            v3 = ix + (iy + 1)*nx
            v4 = ix + 1 + (iy + 1)*nx
            T[2*ix + iy*2*(nx-1), :] = [v1, v2, v4]
            T[2*ix + 1 + iy*2*(nx-1), :] = [v1, v4, v3]

    B = []
    for i in range(nx):
        B.append(i)
        B.append(i + nx*(nx - 1))
    for i in range(1, nx - 1):
        B.append(nx*i)
        B.append(nx - 1 + nx*i)

    return V, T, B

def quadrature(f, K):
    """
    Approximate the integral of f over K using one-point quadrature.
    """
    xy = np.mean(K, axis=0)
    return area(K) * f(xy[0], xy[1])

def make_rhs(f, V, T, B):
    """
    For the right-hand side vector F, where F_i is given by the integral of the
    given function f times the ith basis function, defined on the mesh (V, T).

    The entries in F corresponding to the boundary vertices (listed in B) are
    set to zero.
    """
    N = V.shape[0]
    F = np.zeros(N)

    for it in range(T.shape[0]):
        K = V[T[it,:], :]
        F[T[it,:]] += 1/3 * quadrature(f, K)

    for i in B:
        F[i] = 0.0

    return F

### 2. (3 points)

Write a function `inclusion(nx, B, Bc)`, that forms the matrix representation of the inclusion operator $I_0 : V_0 \hookrightarrow V_h$.

The meshes are assumed to be Cartesian grids, where the square cells are cut along the diagonal from bottom-left to top-right.
The fine mesh has $n_x$ vertices in each dimension ($n_x$ must be odd), and the coarse mesh has $(n_x + 1)/2$ vertices in each dimension.

The matrix representation $J$ of $I_0$ should be of size $n_x^2 \times ((n_x+1)/2)^2$.

The vertices are ordered lexicographically: the index of the vertex at position $(i_x, i_y)$ in the fine grid is given by $i_x + i_y n_x$.
Similarly for the coarse grid.

The input array `B` is a list of fine boundary vertices; these rows of $J$ should be set to zero.
The input arrow `Bc` is a list of coarse boundary vertices; these columns of $J$ should be set to zero.

In [2]:
def inclusion(nx, B, Bc):
    nxc = int(np.ceil(nx/2))
    J = np.zeros((nx*nx, nxc*nxc))
    return J

### 3. (3 points)

Write a function `problem_setup(nx)` that:

1. Assembles the finite element matrix $A$ and right-hand side $b$ (with $f \equiv 1$) on the Cartesian grid with $n_x$ vertices in each dimension.
2. Assembles the finite element matrix $A_0$ on the coarse grid with $(n_x + 1)/2$ vertices in each dimension.
3. Forms the coarse-grid solver $B_0 = I_0 A_0^{-1} I_0^T$.
4. Forms the vertex patch solver $B_D = \sum_{i=1}^J I_i A_i^{-1} I_i^T$. **Note:** $B_D$ is easy to obtain from the fine-grid matrix $A$.
5. Return the tuple $(A, b, B_0, B_D)$.

In [3]:
def problem_setup(nx):
    # Assemble and return (A, b, B_0, B_D)
    pass

### 4. (3 points)

For $n_x \in \{11, 31, 51, 71 \}$, set up the problem using `problem_setup`, and then solve the system using (preconditioned) conjugate gradient (using the `cg` function given below) with

1. No preconditioner (pass `None`).
2. $B_D$ as a preconditioner.
3. $B = B_0 + B_D$ as a preconditioner.

Explain the results in terms of the domain decomposition theory.

In [4]:
import scipy.sparse.linalg as spla

class ItCounter:
    def __init__(self):
        self.it = 0
    def __call__(self, x):
        self.it += 1

def cg(A, b, preconditioner=None):
    counter = ItCounter()
    spla.cg(A, b, M=preconditioner, rtol=1e-7, callback=counter)
    print("CG solved system in {} iterations".format(counter.it))

### 5. (3 points)

Consider the spectrum of the operators $A$, $B_D A$, and $BA$ (with $B = B_0 + B_D$).

For each matrix, what bounds do you expect on the maximum and minimum eigenvalues, and why?